### Logistic Regression

#### Import Packages

In [1]:
%run ml_helpers.ipynb
%run ml.ipynb

/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
866684   11278975      True   522  2018     4    14    8     2   First  0.00   
1329041  11962187     False   824  2019    12    52   27    14  Second  0.00   
1038627  11507658     False   422  2018    11    46   15     2   First  0.10   
1046400  11517935     False  1133  2018    11    48   26    20   Third  0.53   
1102529  11599380     False   512  2019     2     8   18     9  Second  0.09   
886289   11305804     False  1933  2018     5    18    4    21   Third  0.00   
1210250  11759882     False   523  2019     7    29   16     4   First  0.40   
1242348  11806266      True  2511  2019     8    34   25    16   Third  0.00   
1184502  11736538     False   814  2019     6    24   13    23   Third  0.04   
857930   11266794     False  1134  2018     3    13   26     3   First  0.22   
1209878  11758862     False  2433  2019     7    29   15    17   Third  0.00   
1170147  11700968      True   431  2019 

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-1-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
1220086  11773974     False  1821  2019     7    30   28    14  Second  0.00   
1098670  11593589     False  1933  2019     2     7   12    14  Second  0.29   
1265365  11839979     False   215  2019     9    39   25     2   First  0.00   
818915   11214999     False  1933  2018     1     4   25    16   Third  0.00   
1222709  11778884     False  1824  2019     7    31   31    21   Third  0.00   
...           ...       ...   ...   ...   ...   ...  ...   ...     ...   ...   
1313683  11911236     False   612  2019    12    49    4    15  Second  0.00   
891559   11312271     False   513  2018     5    19   11    15  Second  0.03   
1208882  11757387     False   731  2019     7    28   14    11  Second  0.01   
846603   11251814      True   921  2018     3    10    9    16   Third  0.00   
12

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-1-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

         Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  count_l_stops  \
1222465   114  2019      7  Second  0.00   0.0    76    63            0.0   
1274470  1225  2019     10   Third  0.00   0.0    71    48            1.0   
1102593  1925  2019      2  Second  0.09   1.9    31    22            0.0   
1208205   111  2019      7   Third  0.01   0.0    94    73            3.0   
1173472  2524  2019      5   Third  0.12   0.0    80    61            0.0   
...       ...   ...    ...     ...   ...   ...   ...   ...            ...   
1261555  1533  2019      9   Third  0.00   0.0    85    64            1.0   
1095512   111  2019      2   Third  0.50   0.0    46    11            3.0   
1304395  1711  2019     11   Third  0.00   0.0    49    39            0.0   
1193307   634  2019      6  Second  0.00   0.0    87    63            2.0   
1184005   624  2019      6  Second  0.04   0.0    69    52            0.0   

         count_bus_stops  ...  count_restaurants  count_bars  count_daycare

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [2]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

#### Read and Process Data

In [9]:
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest", "Domestic", "Serious"])
data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)

In [4]:
data.dtypes

district               category
sector                 category
Year                      int64
Month                  category
Watch                  category
Beat                   category
count_l_stops           float64
count_bus_stops         float64
count_metra_stops       float64
count_restaurants       float64
count_bars              float64
count_daycares          float64
count_entertainment     float64
count_businesses        float64
road_distance_ft        float64
TOTAL POPULATION        float64
dist_to_police          float64
dist_to_hospital        float64
SNOW                    float64
TMAX                    float64
TMIN                    float64
high_crime              float64
high_crime_geog_lag     float64
dtype: object

#### Setup ML

In [11]:
models = {
    'LogisticRegression': LogisticRegression(), 
}
grid = {
    'LogisticRegression': [{'penalty': x, 'C': y, 'random_state': 0} 
                           for x in ('l2', 'none') \
                           for y in (0.001, 0.01, 0.1, 1, 10, 100, 1000)]
}

In [6]:
data_list = prep_data(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"])

      district sector  Year Month   Watch  Beat  count_l_stops  \
29557       17      1  2018     1   First  1713            3.0   
29558       17      1  2018     1  Second  1713            3.0   
29559       17      1  2018     1   Third  1713            3.0   
29560       17      1  2018     2   First  1713            3.0   
29561       17      1  2018     2  Second  1713            3.0   
...        ...    ...   ...   ...     ...   ...            ...   
49256        3      1  2019    11  Second   312            2.0   
49257        3      1  2019    11   Third   312            2.0   
49258        3      1  2019    12   First   312            2.0   
49259        3      1  2019    12  Second   312            2.0   
49260        3      1  2019    12   Third   312            2.0   

       count_bus_stops  count_metra_stops  count_restaurants  ...  \
29557             26.0                0.0               63.0  ...   
29558             26.0                0.0               63.0  ...   


Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [7]:
logistic_results = average_grid_searches(LogisticRegression(), "LogisticRegression", 
                          grid, 
                          data_list, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]


In [12]:
logistic_results.to_csv("../final_data/log_reg_results.csv")